In [1]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

Using TensorFlow backend.


In [2]:
print(os.getcwd())
#os.chdir('archive/asl_alphabet_train/asl_alphabet_train')
os.listdir()
train_dir = '../jagannathan/ASL_DATA_4K/archive/asl_alphabet_train/asl_alphabet_train/'
test_dir = '../jagannathan/ASL_DATA_4K/archive/asl_alphabet_test/asl_alphabet_test/'

/home/jupyter


In [3]:
from tensorflow.keras.applications import vgg16

# Init the VGG model 
vgg_conv = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(64, 64, 3))

2021-11-19 05:02:42.264255: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2199995000 Hz
2021-11-19 05:02:42.266222: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e71d9e3680 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-19 05:02:42.266246: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-19 05:02:42.268314: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [4]:
# Freeze all the layers
for layer in vgg_conv.layers[:]:
    layer.trainable = False

# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7f97dc659710> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97dc62f4d0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97dc413350> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f97dbb77610> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97dbb92b90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97dc413190> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f97d80d6e90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97d80dc510> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97d80e0990> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7f97d80e5a10> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x7f97dc413f90> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 

In [5]:
# Create the model
model = Sequential()

# Add the vgg convolutional base model
model.add(vgg_conv)

# Add new layers
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(29, activation='softmax'))

# Show a summary of the model. Check the number of trainable parameters
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                29725     
Total params: 16,842,589
Trainable params: 2,127,901
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
# Load the normalized images
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10

# Data generator for training data
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(64, 64),
        batch_size=train_batchsize,
        class_mode='categorical')

# Data generator for validation data
validation_generator = validation_datagen.flow_from_directory(
        test_dir,
        target_size=(64, 64),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 87000 images belonging to 29 classes.
Found 0 images belonging to 0 classes.


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                29725     
Total params: 16,842,589
Trainable params: 2,127,901
Non-trainable params: 14,714,688
_________________________________________________________________


In [8]:
from keras import optimizers
# Configure the model for training
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train the model
history = model.fit(
      train_generator,
      steps_per_epoch=
         train_generator.samples/train_generator.batch_size,
      epochs=2,
      validation_data=validation_generator, 
      validation_steps=
         validation_generator.samples/validation_generator.batch_size)

Epoch 1/2
870/870 [==============================] - 1539s 2s/step - loss: 1.3895 - acc: 0.6558
Epoch 2/2
870/870 [==============================] - 1075s 1s/step - loss: 0.4487 - acc: 0.8939


In [9]:
# Utility function for plotting of the model results
def visualize_results(history):
    # Plot the accuracy and loss curves
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'b', label='Training acc')
    plt.plot(epochs, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'b', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()


# Run the function to illustrate accuracy and loss
visualize_results(history)

KeyError: 'val_acc'

In [10]:
history.history

{'loss': [1.3895189762115479, 0.44872114062309265],
 'acc': [0.6557931303977966, 0.8939425349235535]}